<a href="https://colab.research.google.com/github/LilitKharatyan/MT-Evaluation/blob/main/Arm_EnNMT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Getting Bilingual Text from the Web-pages**

In [ ]:
import urllib.request
import codecs
from bs4 import BeautifulSoup

url="https://www.mfa.am/en/speeches/"
f = urllib.request.urlopen(url)
web=f.read().decode('utf-8')
soup = BeautifulSoup(web, 'lxml')
eng = []
links = soup.find_all('a')
for link in links:
    link=link['href'] 
    if link.startswith(url):
      text=[]
      camps=link.split("/")
      if len(camps)>6:
        eng.append(link)


#out=codecs.open("en.txt","w",encoding="utf-8")

for i in eng:
  print("I",i)
  #You should open a file for each web page. (You may prefer to put the files in different directories
  outfilename=str(i.split("/")[-1])+"-en.txt"
  out=codecs.open(outfilename,"w",encoding="utf-8")
  h = urllib.request.urlopen(i)
  web=h.read().decode('utf-8')
  
  soup = BeautifulSoup(web, 'lxml')

  title = soup.find_all('strong')[0]
  if not title==None:
    title=title.getText()
    print(title)
    out.write(title+"\n")

  entry=soup.findAll("div", {"class" : "static-content"})[0]
  soup2 = BeautifulSoup(str(entry), 'lxml')
  paras=soup2.find_all("p")
  for para in paras:
    para=para.getText()
    print(para)
    out.write(para+"\n")
  out.close()

#aut=codecs.open("arm.txt","w",encoding="utf-8")

for i in eng:
  try:
    print("I",i)
    
    outfilename=str(i.split("/")[-1])+"-am.txt"
    out=codecs.open(outfilename,"w",encoding="utf-8")
    z = urllib.request.urlopen(i)
    web=z.read().decode('utf-8')

    soup = BeautifulSoup(web, 'lxml')

    spanL2=soup.findAll("li", {"class" : "fl pr"})
    soup2 = BeautifulSoup(str(spanL2),'lxml')
    linkL2=soup2.find_all("a")[0]['href'] 


    arm = urllib.request.urlopen(linkL2)
    web=arm.read().decode('utf-8')

    soup = BeautifulSoup(web, 'lxml')

    title = soup.find_all('strong')[0]
    if not title==None:
      title=title.getText()
      print(title)
      out.write(title+"\n")

    entry=soup.findAll("div", {"class" : "static-content"})[0]
    soup2 = BeautifulSoup(str(entry), 'lxml')
    paras=soup2.find_all("p")
    for para in paras:
      para=para.getText()
      print(para)
      out.write(para+"\n")
  except:
    pass

**Getting read of charchters between English and Armenian words**

In [ ]:
with open('baratian.tab.txt') as f:
  lines = f.readlines()
  for n in lines:
    x = re.sub("\(.*?\)|\[.*?\]|\ adv |\ n |\ v |\ a |\ n,|\ I|\ II |\ III | 1.","", n)
    print(x)

"""**Getting only English and Armenian words**"""

with open('baratian.tab.txt') as f:
  lines = f.readlines()
  for i in lines:
    english_word = i.split()[0]
    armenian_word = i.split()[3]
    print(english_word, armenian_wor

**Etracting English-Armenian Monoword Translations from a Tab-Delimited Text File**

In [ ]:
import codecs
import re

inputfile=codecs.open("baratian.tab.txt","r",encoding="utf-8")
outputfile=codecs.open("baratian-converted.txt","w",encoding="utf-8")
for linia in inputfile:
    linia=linia.rstrip()
    #We can separate English word from the rest using split with [ 
    fields=linia.split("[")
    english_word=fields[0].strip()
    #We check whether the word is "uniword"
    if len(english_word.split(" "))==1:
        #now we split the line by n, a, v, adv
        if linia.find("] n ")>-1: fields2=linia.split("] n ")
        elif linia.find("] a ")>-1: fields2=linia.split("] a ")
        elif linia.find("] v ")>-1: fields2=linia.split("] v ")
        elif linia.find("] adv ")>-1: fields2=linia.split("] adv ")
        #the Armenian words are in the second place of this split, but there can be different words and other information separated by ,
        fields3=fields2[1].split(",")
        #we check whether the Armenian word is "monoword"
        for armenian_word in fields3:
            armenian_word=armenian_word.strip()
            if len(armenian_word.split(" "))==1:
                cadena1=english_word+"\t"+armenian_word
                print(cadena1)
                outputfile.write(cadena1+"\n")

**Automated Script for Creating Alignment Script for Hunalign Using Source and Target Language Text Files and Bilingual Dictionary**

In [ ]:
import codecs
import os
import sys
import argparse

parser = argparse.ArgumentParser(description='A script to create the align script to be used with hunalign.')
parser.add_argument("--dirSL", type=str, help="The input dir containing the segmented text files for the source language.", required=True)
parser.add_argument("--dirTL", type=str, help="The input dir containing the segmented text files for the target language.", required=True)
parser.add_argument("--dirALI", type=str, help="The output dir to save the aligned files.", required=True)
parser.add_argument("--dictionary", type=str, help="The bilingual dictionary to use.", required=True)
parser.add_argument("--script", type=str, help="The name of the alignment script.", required=True)
parser.add_argument("--r1", type=str, help="The first string for name replacement.", required=False)
parser.add_argument("--r2", type=str, help="The second string for name replacement.", required=False)
parser.add_argument("--windows", action='store_true', help="Create a bat file for Windows.", required=False)

args = parser.parse_args()

windows=False
if args.windows:
    windows=True
if args.r1:
    r1=args.r1
else:
    r1=""
if args.r2:
    r2=args.r2
else:
    r2=""


dir1=args.dirSL
dir2=args.dirTL
dir3=args.dirALI
if not os.path.exists(dir3):
    os.makedirs(dir3)
outfile=args.script
hundict=args.dictionary

thisdir = os.getcwd()
files1=[]
for r, d, f in os.walk(dir1):
    for file in f:
        files1.append(file)
        
files2=[]
for r, d, f in os.walk(dir2):
    for file in f:
        files2.append(file)
sortida=codecs.open(outfile,"w",encoding="utf-8")
for file1 in files1:
    file2=file1.replace(r1,r2)
    if file2 in files2:
        fileali="ali-"+file1
        if windows:
            cadena="hunalign.exe "+hundict+" -utf -realign -text \""+dir1+"/"+file1+"\" "+"\""+dir2+"/"+file2+"\" > \""+dir3+"/"+fileali+"\""
        else:
            cadena="timeout 5m ./hunalign "+hundict+" -utf -realign -text \""+dir1+"/"+file1+"\" "+"\""+dir2+"/"+file2+"\" > \""+dir3+"/"+fileali+"\""
        print(cadena)
        sortida.write(cadena+"\n")
    else:
        print("***",file1)

**Python Script to Select Hunalign Alignments Above a Specified Confidence and Merge into a Single File**

In [ ]:
import sys
import codecs
import unicodedata
import argparse
import os

def remove_control_characters(s):
    return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")

parser = argparse.ArgumentParser(description='A script to select hunalign aligments in a directory with a higner confidence than the given one into a single file..')
parser.add_argument("--inDir", type=str, help="The input dir containing the hunalign alignments.", required=True)
parser.add_argument("--outFile", type=str, help="The input fule.", required=True)
parser.add_argument("--confidence", type=float, help="The minimun confidence (the output confidence will be highet tnar the one provided..", required=True)


args = parser.parse_args()


dirin=args.inDir
fout=args.outFile
confidence=args.confidence

sortida=codecs.open(fout,"w",encoding="utf-8")

for r, d, f in os.walk(dirin):
    for file in f:
        fullname=os.path.join(dirin,file)
        
        entrada=codecs.open(fullname,"r",encoding="utf-8")

        llistasortida=[]
        iguals=0
        totals=0
        for linia in entrada:
            try:
                linia=linia.rstrip().replace("~~~","")
                camps=linia.split("\t")
                L1seg=camps[0].strip()
                L1seg=" ".join(L1seg.split())
                L2seg=camps[1].strip()
                L2seg=" ".join(L2seg.split())
                totals+=1
                if not L1seg=="<p>" and L1seg==L2seg: iguals+=1
                sim=float(camps[2])
                if sim>=confidence and not L1seg=="<p>" and not L1seg=="" and not L2seg=="<p>" and not L2seg=="":
                    L1seg=remove_control_characters(L1seg)
                    L2seg=remove_control_characters(L2seg)
                    if len(L1seg)>0 and len(L2seg)>0:
                        cadena=L1seg+"\t"+L2seg
                        llistasortida.append(cadena)
            except:
                pass
        try:
            percent=iguals/totals
        except:
            percent=1
        if percent<=0.5:
            for cadena in llistasortida:
                camps=cadena.split("\t")
                sl1=camps[0]
                #print(len(sl1),sl1)
                sl2=camps[1]
                toWrite=True
                if len(sl1)>25 and sl1==sl2:
                    toWrite=False
                if toWrite:
                    sortida.write(cadena+"\n")
        